In [ ]:
import os
import sys
sys.path.append(os.path.curdir)
os.chdir('..')
import core.ssh
import getpass

In [ ]:
Pkeypassword = getpass.getpass("password:")
userName = '' 
key1 = getpass.getpass("key1:")
key2 = getpass.getpass("key2:")
bucket = "ggr-bucket-cbf77f1e-eea2-4b4a-88b2-ae787daf3f42" 
rsa_id_path = ""
folder = ''
mgFileName = ''

In [ ]:
print(core.ssh.printTime(), 'SSH connection established!')
sshSession, workdir = core.ssh.PKeyLogin_Trans(rsa_id_path, '{}'.format(Pkeypassword), '137.226.78.226', 22, userName)

<img src="./214.png" width = "500" height = "400" alt="1" align=left />

In [ ]:
print(core.ssh.printTime(), 'Downloading dataset to the master node...')
core.ssh.k8_3s3download(sshSession, key1, key2, folder) 

<img src="./IMG_1024.JPG" width = "150" height = "100" alt="1" align=left />
<img src="./IMG_1025.JPG" width = "150" height = "100" alt="1" align=left />
<img src="./IMG_1026.JPG" width = "150" height = "100" alt="1" align=left />

In [ ]:
print(core.ssh.printTime(), 'Creating Persistent Volume and meshroom container...')
core.ssh.kubectlApply(sshSession, '{}/wzlk8toolkitCache/k8_3configuration/pvcJobMeshroom.yaml'.format(workdir))

In [ ]:
print(core.ssh.printTime(), 'Getting pod name...')
fullPodName = core.ssh.kubectlGetFullPodName(sshSession, 'yz-meshroom')
print(core.ssh.printTime(), 'Pod name is: ', fullPodName)

In [ ]:
print(core.ssh.printTime(), 'Moving dataset into computation units...')
core.ssh.kubectlCp(sshSession, '{}/wzlk8toolkitCache/{}'.format(workdir, folder), fullPodName, '{}'.format(folder))

In [ ]:
print(core.ssh.printTime(), 'Initiating computing environment...')
_,stdout,_ = sshSession.exec_command('kubectl exec -n ggr {} -- mv /opt/AliceVision_install /opt/AliceVision_bundle'.format(fullPodName))
print(stdout.read().decode())
core.ssh.getstatus(sshSession)

In [ ]:
if mgFileName != '':
    """
    given .mg file should be put in the raw dataset folder and must be generated or
    converted from Meshroom version 2021.1!
    """
    # generate a project .mg file from dataset
    print(core.ssh.printTime(), 'Generating a standard .mg file.')
    _, stdout, _ = sshSession.exec_command('kubectl exec -n ggr {} -- meshroom_batch -i /tmp/{} --compute no --save /tmp/{}/generatedMgTemplate.mg -o /tmp/MeshroomCache'.format(fullPodName, folder, folder))
    print(core.ssh.printTime(), stdout.read().decode())

    # create a shell script in k8-3
    sshSession.exec_command('echo "#!/bin/sh\npython /tmp/{}/mgFileEditor.py /tmp/{}/{} {}" > {}/wzlk8toolkitCache/mgFileTemplates/runpython.sh'.format(folder, folder, mgFileName, folder, workdir, ))

    # copy the shell script into container, this script is used to run python scripts inside container.
    core.ssh.kubectlCp(sshSession, '{}/wzlk8toolkitCache/mgFileTemplates/runpython.sh'.format(workdir), fullPodName, '{}'.format(folder))

    # copy project mgFile into container
    core.ssh.kubectlCp(sshSession, '{}/wzlk8toolkitCache/mgFileTemplates/{}'.format(workdir, mgFileName), fullPodName, '{}'.format(folder))

    print(core.ssh.printTime(), 'Inserting customized computing graph')
    urlToMgFileEditer = 'https://raw.githubusercontent.com/zieft/wzlk8toolkit/master/Scripts/mgFileEditor.py'
    _, stdout, _ = sshSession.exec_command('kubectl exec -n ggr {} -- wget -P /tmp/{} {}'.format(fullPodName, folder, urlToMgFileEditer))
    print(core.ssh.printTime(), stdout.read().decode())

    _, stdout, _ = sshSession.exec_command('kubectl exec -n ggr {} -- bash /tmp/{}/runpython.sh'.format(fullPodName, folder))
    print(core.ssh.printTime(), stdout.read().decode())
    core.ssh.getstatus(sshSession)


    print(core.ssh.printTime(), 'Computing...This process may take hours to finish, please wait...')
    _, stdout, _ = sshSession.exec_command('kubectl exec -n ggr {} -- meshroom_compute --cache /tmp/MeshroomCache /tmp/{}/{}'.format(fullPodName, folder, mgFileName))
    print(core.ssh.printTime(), stdout.read().decode())

else:
    # run a standard pipeline
    print(core.ssh.printTime(), 'Computing...Please wait...')
    _,stdout,_ = sshSession.exec_command('kubectl exec -n ggr {} -- meshroom_batch -i /tmp/{} -o /tmp/MeshroomCache/{}_out'.format(fullPodName, folder, folder))
    print(stdout.read().decode())
    core.ssh.getstatus(sshSession)

<img src="./521.png" width = "500" height = "400" alt="1" align=left />

In [ ]:
print(core.ssh.printTime(), 'Dismounting finished Job...Please wait...')
core.ssh.kubectlDelete(sshSession, 'job', 'yz-meshroom')

In [ ]:
# create a minio server in the cluster, expose port 9000 as service
print(core.ssh.printTime(), 'Creating minIO Service... Waiting...')
core.ssh.kubectlApply(sshSession, '{}/wzlk8toolkitCache/k8_3configuration/svcForUpload.yaml'.format(workdir))

In [ ]:
# Get IP address of the host machine from minIO service
svcIP = core.ssh.getSvcIp(sshSession)

In [ ]:
# create an alias in minio client on master node.
print(core.ssh.printTime(), 'Creating myminio in Minio Client...')
core.ssh.mcAliasSet(sshSession, workdir, 'myminio', svcIP, key1, key2, 's3v4')

In [ ]:
# upload output data to s3 storage
print(core.ssh.printTime(), 'Uploading output data to S3 Storage...')
pathJob = 'MeshroomCache'
core.ssh.mcUpload(sshSession, key1, key2, pathJob, bucket, folder)

In [ ]:
print(core.ssh.printTime(), 'Job finished, clearing cache...')
core.ssh.kubectlDelete(sshSession, 'job', 'minio-server')
core.ssh.kubectlDelete(sshSession, 'service', 'minio-service')
core.ssh.kubectlDelete(sshSession, 'persistentvolumeclaim', 'yz-pvc')
# remove cache folders/scripts in k8-3
sshSession.exec_command('rm -rf wzlk8toolkitCache')
core.ssh.getstatus(sshSession)

# close session
sshSession.close()
print(core.ssh.printTime(), 'All done!')